In [179]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.types import *
#from pyspark.sql.window import Window

import getpass

In [ ]:
userName = getpass.getuser()
print(userName)
id_user = '1000'

In [ ]:
spark = SparkSession.\
        builder.\
        config("spark.ui.port","0").\
        master("yarn").\
        enableHiveSupport().\
        config("spark.sql.warehouse.dir", f"/user/{userName}/warehouse").\
        appName("Exercises").\
        getOrCreate()

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", "2")

In [ ]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

In [ ]:
# exercise 1:
path = "/public/crime/csv/Crimes.csv"

In [ ]:
'''schema = StructType().\
        add('ID', IntegerType(), True).\
        add('Case Number', StringType(), True).\
        add('Date', StringType(), True).\
        add('Block', StringType(), True).\
        add('IUCR', StringType(), True).\
        add('Primary Type', StringType(), True).\
        add('Description', StringType(), True).\
        add('Location Description' ,StringType(), True).\
        add('Arrest', BooleanType(), True).\
        add('Domestic', BooleanType(), True).\
        add('Beat', StringType(), True).\
        add('District', StringType(), True).\
        add('Ward', IntegerType(), True).\
        add('Community Area', IntegerType(), True).\
        add('FBI code', StringType(), True).\
        add('X Coordinate', LongType(), True).\
        add('Y Coordinate', LongType(), True).\
        add('Year', StringType(), True).\
        add('Updated On', StringType(), True).\
        add('Latitude', FloatType(), True).\
        add('Longitude', FloatType(), True).\
        add('Location', StringType(), True)
crimes = spark.read.format('csv').schema(schema).option('header', True).option('inferschema', True).load(path)
'''

In [ ]:
crimes= spark.read.csv(path, header=True, inferSchema=True)

In [ ]:
crimes.printSchema()

In [ ]:
crimes.show(5)

In [ ]:
# Get monthly count of primary crime type

#crimes.select(split(col("Date"), "/")[1].alias("month"), split(split(col("Date"), "/")[2], ' ').alias("year")).show(20)
tmp_crimes = crimes.withColumn('Date', split(col('Date'), ' ')[0])

In [ ]:
print(type(tmp_crimes))

In [ ]:
# sap' xep' theo month tang dan`, count_primary_type giam? dan`
tmp = tmp_crimes.groupBy('Date').agg(count('Primary Type').alias('count_primary_type')).sort(asc(split(col('Date'), '/')[0]), desc('count_primary_type'))

In [ ]:
tmp.show(5)

In [ ]:
tmp = tmp.withColumn('count_primary_type', col('count_primary_type').cast('STRING'))

In [ ]:
tmp.show(5)

In [ ]:
text_path = "/user/exercise/solutions/solution1/crimes_by_type_by_month"

In [ ]:
head = tmp.select(concat_ws(',', col('Date'), col('count_primary_type')).alias(f'{tmp.columns[0]},{tmp.columns[1]}'))

In [ ]:
head.show(5)

In [ ]:
head.write.text(path=text_path, compression='gzip')

In [ ]:
#-------------------------------------------ex2----------------------------------#

In [130]:
def extract_txt(path, headers):
    text_data = spark.sparkContext.textFile(path)
    context = text_data.map(lambda k: k.split(","))
    return context.toDF(headers)

In [131]:
path = "/public/retail_db/orders/part-00000"
headers = ['order_id', 'order_date', 'order_customer_id', 'order_status']
orders = extract_txt(path, headers)

In [132]:
orders.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)



In [133]:
orders.show(5)

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|
+--------+--------------------+-----------------+---------------+
only showing top 5 rows



In [ ]:
spark.sql("DESCRIBE TABLE customers")

In [139]:
spark.catalog.setCurrentDatabase('band_retail')

In [141]:
customers = spark.sql("SELECT * FROM customers")

In [142]:
customers.printSchema()

root
 |-- customer_city: string (nullable = true)
 |-- customer_email: string (nullable = true)
 |-- customer_fname: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- customer_lname: string (nullable = true)
 |-- customer_password: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_street: string (nullable = true)
 |-- customer_zipcode: string (nullable = true)



In [155]:
customers = spark.sql("SELECT * FROM customers")

In [ ]:
customers.show()

In [168]:
filter_not_order = orders.join(customers, on=customers.customer_id != orders.order_customer_id).sort('customer_lname', 'customer_fname')

In [177]:
counting = crimes.groupby('Primary Type').count().sort(desc('count'))

In [178]:
counting.show(5)

+---------------+-------+
|   Primary Type|  count|
+---------------+-------+
|          THEFT|1570224|
|        BATTERY|1368734|
|CRIMINAL DAMAGE| 850873|
|      NARCOTICS| 740084|
|        ASSAULT| 479748|
+---------------+-------+
only showing top 5 rows



In [173]:
counting.printSchema()

root
 |-- Primary Type: string (nullable = true)
 |-- count: long (nullable = false)



In [182]:
#--------------------ex4---------------------------------------#